In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [01:18<00:00, 32.4MB/s]
100% 2.29G/2.29G [01:18<00:00, 31.2MB/s]


In [3]:
!unzip -q "/content/chest-xray-pneumonia.zip"


In [9]:
!apt-get install texlive-latex-recommended texlive-fonts-recommended texlive-latex-extra > /dev/null 2>&1


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import requests
import json
import os
from datetime import datetime
import subprocess

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')

# Configuration
GROQ_API_KEY = "gsk_REDACTED"
MODEL_PATH = "/content/drive/MyDrive/hafsa-model.h5"
LATEX_TEMPLATE = """\\documentclass{{article}}
\\usepackage[utf8]{{inputenc}}
\\title{{Medical Report}}
\\date{{{date}}}

\\begin{{document}}
\\maketitle

\\section{{Diagnosis Results}}
{diagnosis}

\\section{{Radiology Findings}}
{findings}

\\section{{Recommendations}}
\\begin{{itemize}}
{recommendations}
\\end{{itemize}}

\\section{{Prescription Guidance}}
{treatment}

\\end{{document}}
"""

def load_model():
    """Load the pre-trained pneumonia detection model"""
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Model not found at {MODEL_PATH}")
    return tf.keras.models.load_model(MODEL_PATH)

def predict_image(model, image_path):
    """Make prediction on an image"""
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array, verbose=0)[0][0]
    return "Pneumonia" if prediction >= 0.5 else "Normal", float(prediction)

def generate_ai_report(diagnosis, confidence):
    """Generate medical report using Groq API with detailed medical prompt"""
    prompt = f"""Act as a senior radiologist. Generate a detailed medical report for a chest X-ray diagnosis in JSON format.

Patient Diagnosis: {diagnosis} (Confidence Score: {confidence:.2f})

Required JSON structure:
{{
    "findings": "Detailed radiological observations including lung fields, opacities, and other notable features",
    "clinical_interpretation": "Medical analysis of the findings in clinical context",
    "recommendations": [
        "Prioritized list of clinical recommendations",
        "Imaging follow-up suggestions",
        "Referral suggestions if needed"
    ],
    "medication": {{
        "antibiotics": ["Drug name, dosage, and duration if bacterial pneumonia suspected"],
        "antivirals": ["Options if viral etiology suspected"],
        "symptomatic_relief": ["Fever reducers, cough suppressants"]
    }},
    "follow_up_plan": {{
        "imaging": "Timing for repeat imaging if needed",
        "clinical_review": "Recommended timeline for clinical follow-up",
        "warning_signs": "Red flags requiring urgent attention"
    }},
    "patient_advice": "Lifestyle recommendations and home care instructions"
}}

Example for Pneumonia diagnosis:
{{
    "findings": "Consolidation with air bronchograms in the right lower lobe, associated pleural effusion...",
    "clinical_interpretation": "These findings are consistent with community-acquired pneumonia...",
    "recommendations": [
        "Immediate initiation of empirical antibiotic therapy",
        "Chest CT scan if no improvement in 48 hours",
        "Pulmonology referral for recurrent cases"
    ],
    "medication": {{
        "antibiotics": ["Amoxicillin-clavulanate 875/125 mg every 12 hours for 7 days"],
        "antivirals": ["Oseltamivir 75 mg twice daily if influenza suspected"],
        "symptomatic_relief": ["Paracetamol 1g every 6 hours for fever"]
    }},
    "follow_up_plan": {{
        "imaging": "Repeat X-ray in 4-6 weeks if unresolved",
        "clinical_review": "48-hour follow-up for treatment response",
        "warning_signs": "Dyspnea, hemoptysis, or persistent fever >39°C"
    }},
    "patient_advice": "Adequate hydration, rest, and monitor oxygen saturation"
}}"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "mixtral-8x7b-32768",
        "messages": [{
            "role": "system",
            "content": "You are a medical expert specializing in radiology and pulmonology. Provide structured, clinically valid recommendations."
        },{
            "role": "user",
            "content": prompt
        }],
        "temperature": 0.3,
        "response_format": {"type": "json_object"},
        "max_tokens": 1500
    }

    try:
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status()

        # Extract and clean JSON response
        content_str = response.json()['choices'][0]['message']['content']
        content_str = content_str[content_str.find('{'):content_str.rfind('}')+1]

        return json.loads(content_str)

    except Exception as e:
        print(f"API Error: {str(e)}")
        # Return safe default structure
        return {
            "findings": "Radiological assessment unavailable",
            "clinical_interpretation": "Clinical correlation recommended",
            "recommendations": ["Urgent clinical review required"],
            "medication": {"antibiotics": [], "antivirals": [], "symptomatic_relief": []},
            "follow_up_plan": {
                "imaging": "Repeat X-ray as clinically indicated",
                "clinical_review": "Within 24 hours",
                "warning_signs": "Seek emergency care for breathing difficulties"
            },
            "patient_advice": "Follow up with your primary care physician immediately"
        }

def create_pdf_report(data, output_dir="reports"):
    """Generate PDF using LaTeX template"""
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"medical_report_{timestamp}"

    # Format LaTeX content
    latex_content = LATEX_TEMPLATE.format(
        date=datetime.now().strftime("%Y-%m-%d"),
        diagnosis=f"Condition: {data['diagnosis']} (Confidence: {data['confidence']:.2%})",
        findings=data.get('findings', 'No findings available'),
        recommendations="\n".join([f"\\item {item}" for item in data.get('recommendations', [])]),
        treatment=data.get('treatment', 'No specific treatment recommended')
    )

    # Write LaTeX file
    with open(f"{filename}.tex", "w") as f:
        f.write(latex_content)

    # Compile to PDF using Colab's LaTeX installation
    try:
        subprocess.run(
            ["pdflatex", "-interaction=nonstopmode", f"{filename}.tex"],
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
    except Exception as e:
        print(f"PDF generation warning: {str(e)}")

    # Move PDF to reports directory and clean up
    os.rename(f"{filename}.pdf", f"{output_dir}/{filename}.pdf")
    for ext in [".tex", ".aux", ".log"]:
        if os.path.exists(f"{filename}{ext}"):
            os.remove(f"{filename}{ext}")

    return f"{output_dir}/{filename}.pdf"

def main(image_path):
    """Main workflow"""
    try:
        # Load model and predict
        model = load_model()
        diagnosis, confidence = predict_image(model, image_path)

        # Generate AI report
        ai_data = generate_ai_report(diagnosis, confidence)
        ai_data.update({
            "diagnosis": diagnosis,
            "confidence": confidence
        })

        # Create PDF
        pdf_path = create_pdf_report(ai_data)
        print(f"Report generated at: {pdf_path}")
        return pdf_path

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

if __name__ == "__main__":
    # Example usage
    image_path = "/content/chest_xray/test/PNEUMONIA/person100_bacteria_477.jpeg"
    main(image_path)

Report generated at: reports/medical_report_20250308_103301.pdf
